# Hypothesis testing problems

## Exercise 1

You are a nutritionist researching two different types of diets to see if there is a significant difference in weight loss after one month. You choose two random groups of people; one group follows the first diet, and the other follows the second. At the end of the month, you record the weight loss (in kg) for each person:

| Diet 1 | Diet 2 |
|:-------|:-------|
| 2.0 | 3.0 |
| 2.5 | 3.2 |
| 3.0 | 3.1 |
| 2.8 | 2.9 |
| 2.3 | 2.8 |
| 2.7 | 3.0 |
| 2.5 | 3.2 |

With these data, it seeks to answer the following question: Is there a significant difference in average weight loss between people who followed the first diet and those who followed the second diet?

To draw conclusions, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the test to test the hypothesis. You can use a Student's t-test.
- Analyze the conclusions.

**Hipótesis**
- H₀: μ₁ = μ₂ (no hay diferencia en la media de pérdida de peso).
- H₁: μ₁ ≠ μ₂ (sí hay diferencia).

**Plan**
1. Comprobar supuestos (normalidad y homogeneidad de varianzas).
2. Realizar Student's t-test independiente.
3. Interpretar p-value y medida de efecto.

In [13]:
import numpy as np
from scipy import stats

In [14]:
diet1 = np.array([2.0, 2.5, 3.0, 2.8, 2.3, 2.7, 2.5])
diet2 = np.array([3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2])


In [ ]:
# 1. Supuestos: normalidad y homogeneidad de varianzas

n1 = stats.shapiro(diet1)
n2 = stats.shapiro(diet2)
lev = stats.levene(diet1, diet2)

print("Normalidad Diet1: W={:.3f}, p={:.3f}".format(n1.statistic, n1.pvalue))
print("Normalidad Diet2: W={:.3f}, p={:.3f}".format(n2.statistic, n2.pvalue))
print("Homogeneidad de varianzas: W={:.3f}, p={:.3f}".format(lev.statistic, lev.pvalue))

# 2. t-test (usando equal_var según la homogeneidad de varianzas)

t_stat, p_val = stats.ttest_ind(diet1, diet2, equal_var=(lev.pvalue > 0.05))
print("t-statistic: {:.3f}".format(t_stat))
print("p-valor: {:.4f}".format(p_val))

# 3. Interpretacion de p-value y medida de efecto


Normalidad Diet1: W=0.981, p=0.964
Normalidad Diet2: W=0.935, p=0.591
Homogeneidad de varianzas: W=2.271, p=0.158
t-statistic: -3.538
p-valor: 0.0041
p-value de (Diet1 - Diet2): -1.891


La probabilidad de observar una diferencia tan grande entre las medias por puro azar es extremadamente baja.

Por lo tanto, rechazamos la hipótesis nula (H₀).

Existe evidencia estadísticamente significativa de que las dos dietas generan pérdidas de peso distintas.

La Dieta 2 produce una mayor pérdida de peso promedio que la Dieta 1.

Para comprender si la diferencia es relevante en la práctica, calculamos Cohen’s d:

Un valor alrededor de d ≈ 2 (aprox.) indica un efecto muy grande.

Esto significa que la diferencia no solo es significativa, sino que además es muy notable en magnitud.

Conclusiones:

Hay una diferencia clara y contundente entre ambas dietas:

- La Dieta 2 supera consistentemente a la Dieta 1 en pérdida de peso.
- El resultado es estadísticamente significativo y de fuerte impacto práctico.

## Exercise 2

A farmer decides to test three different types of fertilizers to determine if one is superior in terms of corn production. The farmer plants corn on 15 identical plots and uses all three fertilizers (5 plots for each type). At the end of the season, he measures the corn yield (in kg) of each plot, with the following result:

| Fertilizer 1 | Fertilizer 2 | Fertilizer 3 |
|:-------------|:-------------|:-------------|
| 20 | 22 | 24 |
| 21 | 21 | 23 |
| 20 | 23 | 22 |
| 19 | 22 | 23 |
| 20 | 21 | 24 |

With this data, he seeks to answer the following question: Is there a significant difference in average corn yield between the three types of fertilizers?

To help you, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the ANOVA test.
- Analyze the conclusions.
- If one fertilizer is better than another, how can we know it?

### Ejercicio 2 — ¿Difieren los rendimientos medios entre los tres fertilizantes?

**Hipótesis**
- H₀: μ₁ = μ₂ = μ₃ (las medias son iguales).
- H₁: Al menos una media difiere.

**Plan**
1. Comprobar supuestos (normalidad por grupo y homogeneidad de varianzas).
2. Realizar ANOVA one-way.
3. Si ANOVA significativa → realizar post-hoc Tukey para identificar pares que difieren.
4. Calcular medida de efecto (eta-squared).


In [16]:
# ANOVA + Tukey

import numpy as np
from scipy import stats
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

fert1 = [20, 21, 20, 19, 20]
fert2 = [22, 21, 23, 22, 21]
fert3 = [24, 23, 22, 23, 24]

# 1. Supuestos: normalidad por grupo y homogeneidad de varianzas

print("Normalidad Fert1:", stats.shapiro(fert1))
print("Normalidad Fert2:", stats.shapiro(fert2))
print("Normalidad Fert3:", stats.shapiro(fert3))
print("Homogeneidad de varianza (3 grupos):", stats.levene(fert1, fert2, fert3))

# 2. ANOVA one-way

f_stat, p_val = stats.f_oneway(fert1, fert2, fert3)
print("ANOVA: F = {:.3f}, p = {:.4f}".format(f_stat, p_val))


# 3. Post-hoc Tukey (si ANOVA significativa)

data = np.concatenate([fert1, fert2, fert3])
labels = ["F1"]*len(fert1) + ["F2"]*len(fert2) + ["F3"]*len(fert3)
tukey = pairwise_tukeyhsd(endog=data, groups=labels, alpha=0.05)
print("Resultados de Tukey HSD:")
print(tukey.summary())

# 4. Medida de efecto (Eta-squared)

grand_mean = np.mean(data)  # Media total

# Suma de cuadrados entre grupos

ss_between = (
    len(fert1) * (np.mean(fert1) - grand_mean)**2 +
    len(fert2) * (np.mean(fert2) - grand_mean)**2 +
    len(fert3) * (np.mean(fert3) - grand_mean)**2
)

# Suma total de cuadrados

ss_total = np.sum((data - grand_mean)**2)

# Eta squared

eta_squared = ss_between / ss_total

print("Eta squared (η²):", eta_squared)


Normalidad Fert1: ShapiroResult(statistic=np.float64(0.8834906936019853), pvalue=np.float64(0.32542950716414576))
Normalidad Fert2: ShapiroResult(statistic=np.float64(0.8810376385817489), pvalue=np.float64(0.3140395561440371))
Normalidad Fert3: ShapiroResult(statistic=np.float64(0.8810376385817489), pvalue=np.float64(0.3140395561440371))
Homogeneidad de varianza (3 grupos): LeveneResult(statistic=np.float64(0.22222222222222207), pvalue=np.float64(0.8039599174006208))
ANOVA: F = 20.316, p = 0.0001
Resultados de Tukey HSD:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    F1     F2      1.8 0.0099 0.4572 3.1428   True
    F1     F3      3.2 0.0001 1.8572 4.5428   True
    F2     F3      1.4 0.0409 0.0572 2.7428   True
--------------------------------------------------
Eta squared (η²): 0.7719999999999995


**Resultados:**
- Shapiro por grupo: p > 0.05 → no violación de normalidad.
- Levene (3 grupos): p > 0.05 → homogeneidad de varianzas aceptable.
- ANOVA: F ≈ 20.316, p ≈ 0.0001 → p < 0.05, rechazamos H₀ (hay diferencias entre medias).
- Eta-squared ≈ 0.772 → efecto muy grande (≈77% de la variabilidad explicada por el tipo de fertilizante).

**Tukey HSD (post-hoc):**
- F1 vs F2: meandiff = 1.8, p-adj ≈ 0.0099 → diferencia significativa (F2 > F1).
- F1 vs F3: meandiff = 3.2, p-adj ≈ 0.0001 → diferencia significativa (F3 > F1).
- F2 vs F3: meandiff = 1.4, p-adj ≈ 0.0409 → diferencia significativa (F3 > F2).

**Conclusión práctica:** Fertilizante 3 produce el mayor rendimiento promedio, seguido por Fertilizante 2 y luego Fertilizante 1; las diferencias entre los tres son estadísticamente significativas bajo α = 0.05.
